In [ ]:
import os
import sys
import re
sys.path.insert(0, '/mind_data/fongc2/pathology_report_segmentation/')
sys.path.insert(0, '/mind_data/fongc2/pathology_report_segmentation/annotations/')
sys.path.insert(0, '/mind_data/fongc2/data-curation/')
import pandas as pd
import numpy as np
from utils_pathology import save_appended_df
import constants_darwin_pathology as c_dar

from pathology_parse_heme_section_periph_blood import ParseHemePathologySectionPeriphBlood

from utils import set_debug_console, print_df_without_index, drop_cols, mrn_zero_pad, convert_to_int

%config Completer.use_jedi = False


In [ ]:
set_debug_console()


In [ ]:
fname = '/mind_data/minio_data/cdm-data/pathology/table_pathology_heme_notes_parsed.csv'
df = pd.read_csv(fname, header=0, low_memory=False, sep=',')
df = mrn_zero_pad(df=df,col_mrn='MRN')

In [ ]:
## print(df1.loc[df1['ACCESSION_NUMBER'] == 'H20-4400', 'PATH_REPORT_NOTE'].iloc[0])
print(df['PERIPHERAL_BLOOD'].iloc[9])

In [ ]:
obj_p = ParseHemePathologySectionPeriphBlood(pathname=c_dar.pathname, 
                                             fname_path_clean=c_dar.fname_darwin_path_heme,
                                   fname_save='test.csv')

In [ ]:
t = obj_p.return_output()
t.loc[9]

In [ ]:
df.notnull().sum()/df.shape[0]



In [ ]:
print(df['BONE_MARROW_ASPIRATE_SMEAR'].loc[3])



In [ ]:
dict_input = {'Cellularity:': 'CELLULARITY',
              'Quality:': 'QUALITY',
             'Myeloid lineage:': 'MYELOID_LINEAGE',
             'Erythroid lineage:': 'ERYTHROID_LINEAGE',
             'Megakaryocytes:': 'MEGAKARYOCYTES',
             'Plasma cells:': 'PLASMA_CELLS',
             'Congo red:': 'CONGO_RED',
             'Aspirate differential:': 'ASPIRATE_DIFFERENTIAL',
             'Blasts': 'BLASTS',
             'Promyelocytes': 'PROMYELOCYTES',
             'Myelocytes': 'MYELOCYTES',
             'Metamyelocytes': 'METAMYELOCYTES1',
              'Meta': 'METAMYELOCYTES2',
              'Band/PMN Neutrophils': 'NEUTROPHILS_BANDS2',
             'Neutrophils/Bands': 'NEUTROPHILS_BANDS1',
             'Monocytes': 'MONOCYTES',
             'Eosinophils': 'EOSINOPHILS',
             'Basophils': 'BASOPHILS',
             'Erythroid Precursors': 'ERYTHROID_PRECURSORS',
             'Plasma Cells': 'PLASMA_CELLS',
             'Lymphocytes': 'LYMPHOCYTES',
              'Nucleated RBC': 'NUCLEATED_RBC',
             'Number of Cells Counted': 'NUMBER_OF_CELLS_COUNTED',
              'Clot section': 'CLOT_SECTION',
             'M: E Ratio': 'ME_RATIO1',
             'M:E Ratio': 'ME_RATIO2'}

dict_clean = {'ME_RATIO': ['ME_RATIO2', 'ME_RATIO1'],
             'METAMYELOCYTES': ['METAMYELOCYTES1', 'METAMYELOCYTES2'],
             'NEUTROPHILS_BANDS': ['NEUTROPHILS_BANDS1', 'NEUTROPHILS_BANDS2']}



In [ ]:
def _get_attribute(series, str_attr):
    str_reg = '(?<=' + str_attr + ').*?(?=\n)'
    extract_current = series.str.findall(str_reg)
    extract_current = extract_current[extract_current.notnull()]
    extract_current_len = extract_current.apply(lambda x: len(x))
    extract_current_str = extract_current[extract_current_len > 0].apply(lambda x: x[0].strip())
    
    return extract_current_str


In [ ]:
for current_attr, col_name in dict_input.items():
    series_parsed = _get_attribute(series=series, str_attr=current_attr)
    kwargs = {col_name : series_parsed}
    df = df.assign(**kwargs)
    

In [ ]:
for col_name, list_col_name in dict_clean.items():
    series_comb = df[list_col_name[0]].fillna(df[list_col_name[1]])
    kwargs = {col_name : series_comb}
    df = df.assign(**kwargs)
    df = df.drop(columns=list_col_name)

In [ ]:
# df[['ACCESSION_NUMBER', 'BONE_MARROW_BIOPSY'] +  list(dict_input.values())].head(5)



In [ ]:
an = 'H17-431'
print(df.loc[df['ACCESSION_NUMBER'] == an, 'BONE_MARROW_BIOPSY'].iloc[0])
# print(df.loc[df['ACCESSION_NUMBER'] == an, 'PERIPHERAL_BLOOD'].iloc[0])


In [ ]:
df.notnull().sum()



In [ ]:
df.notnull().sum()